In [ ]:
# Simple script to create a kinesis stream
# create-stream.py
import boto3

client = boto3.client('kinesis')
response = client.create_stream(
   StreamName='twitter',
   ShardCount=1
)

In [ ]:
response = client.delete_stream(
   StreamName='twitter'
)

In [ ]:
## Example to use twitter api and feed data into kinesis

from TwitterAPI import TwitterAPI
import boto3
import json
import twitterCreds

## twitter credentials

# Twitter security credentials 
# ACCESS_TOKEN    = "908803963557941248-sRHYClIfMteyPMnwF4hWkARyuHNkJRT"
# ACCESS_SECRET   = "FgGi0GshGh8Xbi0Tmkbks0G4Jvd20J5tTThCLJzxd0UVB"
# CONSUMER_KEY    = "KZT7UkCSyLhVO18Wqx6OJISDY"
# CONSUMER_SECRET = "X6hfBxJZz3jLqo8VeX451d7zW8u8v6yDqpiWTUWoq7hnGQTrp2"
consumer_key = twitterCreds.consumer_key
consumer_secret = twitterCreds.consumer_secret
access_token_key = twitterCreds.access_token_key
access_token_secret = twitterCreds.access_token_secret

api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

r = api.request('statuses/filter', {'locations':'-90,-90,90,90'})

for item in r:
    client.put_record(StreamName="twitter", Data=json.dumps(item), PartitionKey="filler")

In [ ]:
## Example to use twitter api and feed data into kinesis

from TwitterAPI import TwitterAPI
import boto3
import json
import twitterCreds

## twitter credentials

consumer_key = twitterCreds.consumer_key
consumer_secret = twitterCreds.consumer_secret
access_token_key = twitterCreds.access_token_key
access_token_secret = twitterCreds.access_token_secret

api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

kinesis = boto3.client('kinesis')

r = api.request('statuses/filter', {'locations':'-90,-90,90,90'})
# print(r)
tweets = []
count = 0
for item in r:
    jsonItem = json.dumps(item)
    tweets.append({'Data':jsonItem, 'PartitionKey':"filler"})
    count += 1
    if count == 100:
        print(count)
        kinesis.put_records(StreamName="twitter", Records=tweets)
        print(tweets[1])
        count = 0
        tweets = []